In [1]:
import pandas as pd
import openpyxl

In [2]:
df = pd.read_excel("GEMS Schedule.xlsx")

In [3]:
df

,Perner,Name,WorkStatus,Bid Pool Name,Seniority Date,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
0,158520,Craig M Florence,Casual Regular,TSR MK Be Our Guest Server,2005-06-09,2,1,1,1,2,2,2
1,199716,Rebecca A Epley Cunningha,Casual Regular,TSR MK Be Our Guest Server,2005-10-18,2,1,1,2,2,2,2
2,90075,Moe Chehab,Casual Regular,TSR MK Be Our Guest Server,2006-07-18,2,2,2,1,1,1,2
3,92156134,PATRICK E Gosy,Casual Regular,TSR MK Be Our Guest Server,2007-09-13,2,1,1,1,2,2,2
4,92055368,Becky R Desenti,Casual Regular,TSR MK Be Our Guest Server,2007-11-04,2,2,1,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
389,92109217,Gary S Bogosian,Casual Regular,TSR MK Tonys Server,2023-05-20,2,1,1,1,2,2,2
390,1210531,Victoria A Bonfiglio,Casual Regular,TSR MK Tonys Server,2023-06-18,2,1,1,2,2,1,2
391,851415,Andre K S Ong,Casual Regular,TSR MK Tonys Server,2023-07-08,1,2,2,2,2,1,1
392,165472,Kim Anne Chennoufi,Casual Regular,TSR MK Tonys Server,2024-07-21,2,1,2,1,1,2,2


In [4]:
# List your day-of-week columns in the correct order.
day_cols = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]

# Convert each day’s numeric value to a string, then join them.
df["Day_Combination"] = df[day_cols].astype(str).agg(''.join, axis=1)

# Now df has a new column "Day_Combination" that might look like "1121121", "1111112", etc.


In [5]:
grouped = df.groupby(["Bid Pool Name", "Day_Combination"])["Perner"].count().reset_index()

# Rename the count column if you like
grouped = grouped.rename(columns={"CM ID": "Num_of_CMs"})

# grouped DataFrame now has:
#   Bid Area   | Day_Combination | Num_of_CMs
# ------------------------------------------
#   Area A     | 1121121         | X
#   Area A     | 1111112         | Y
#   Area B     | 2221111         | Z
#   ...        |      ...        | ...


In [6]:
print(grouped)


                  Bid Pool Name Day_Combination  Perner
0    TSR MK Be Our Guest Server         1112222      11
1    TSR MK Be Our Guest Server         1121222       1
2    TSR MK Be Our Guest Server         1122122       2
3    TSR MK Be Our Guest Server         1122221       6
4    TSR MK Be Our Guest Server         1211222       1
..                          ...             ...     ...
107         TSR MK Tonys Server         2221112       6
108         TSR MK Tonys Server         2221121       2
109         TSR MK Tonys Server         2221122       1
110         TSR MK Tonys Server         2221211       1
111         TSR MK Tonys Server         2222111       7

[112 rows x 3 columns]


In [7]:
#grouped.to_excel('\\\\vi240c060002.woc.prod\\e$\\data\\Day_Combos.xlsx', index=False)

In [8]:
# Sort the main DataFrame by Bid Area, then by Seniority Date ascending
df_sorted = df.sort_values(by=["Bid Pool Name", "Seniority Date"])

# Now for each Bid Area, you have employees listed in seniority order along with their Day_Combination
# You could do something like:
for area, group_data in df_sorted.groupby("Bid Pool Name"):
    print(f"=== {area} ===")
    for idx, row in group_data.iterrows():
        cm_id = row["Perner"]
        seniority_date = row["Seniority Date"]
        pattern = row["Day_Combination"]
        print(f"Perner: {cm_id}, Seniority: {seniority_date}, Days Pattern: {pattern}")
    print()


=== TSR MK Be Our Guest Server ===
Perner: 158520, Seniority: 2005-06-09 00:00:00, Days Pattern: 2111222
Perner: 199716, Seniority: 2005-10-18 00:00:00, Days Pattern: 2112222
Perner: 90075, Seniority: 2006-07-18 00:00:00, Days Pattern: 2221112
Perner: 92156134, Seniority: 2007-09-13 00:00:00, Days Pattern: 2111222
Perner: 92055368, Seniority: 2007-11-04 00:00:00, Days Pattern: 2211122
Perner: 79057, Seniority: 2007-11-25 00:00:00, Days Pattern: 1222211
Perner: 92209439, Seniority: 2009-04-05 00:00:00, Days Pattern: 2112221
Perner: 530508, Seniority: 2009-11-03 00:00:00, Days Pattern: 2111222
Perner: 94325, Seniority: 2009-12-08 00:00:00, Days Pattern: 2221112
Perner: 92056435, Seniority: 2010-06-13 00:00:00, Days Pattern: 2221112
Perner: 297792, Seniority: 2010-06-27 00:00:00, Days Pattern: 1121222
Perner: 500813, Seniority: 2010-07-18 00:00:00, Days Pattern: 1222211
Perner: 570571, Seniority: 2011-03-20 00:00:00, Days Pattern: 2222111
Perner: 630800, Seniority: 2011-04-14 00:00:00, Da

In [9]:
def load_and_prepare_data(excel_file: str) -> pd.DataFrame:
    # Read Excel
    df = pd.read_excel(excel_file)
    
    # List of day columns in your desired order
    day_cols = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    
    # Create a single string representing each row's day pattern, e.g. "1112222"
    df["Day_Combination"] = df[day_cols].astype(str).agg(''.join, axis=1)
    
    return df



In [10]:
def pattern_to_named_days(pattern_str: str, days: list[str]) -> str:
    """
    Given a pattern like '1112222' and a list of day names,
    return a comma-separated string of days for which pattern == '1'.
    """
    named_days = []
    for char, day_name in zip(pattern_str, days):
        if char == '1':  # 1 means working
            named_days.append(day_name)
    return ", ".join(named_days)


In [11]:
def add_readable_day_combinations(df: pd.DataFrame) -> pd.DataFrame:
    # Same day column order you used before
    day_cols = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    
    # Create a new column for the textual version (e.g. "Sunday, Monday, Tuesday")
    df["Day_Combination_Text"] = df["Day_Combination"].apply(
        lambda pattern: pattern_to_named_days(pattern, day_cols)
    )
    
    return df


In [15]:
import pandas as pd

def pattern_to_named_days(pattern_str: str, days: list[str]) -> str:
    """
    Convert a day pattern like '1112222' to a string of day names,
    e.g. 'Sunday, Monday, Tuesday'.
    """
    named_days = []
    for char, day_name in zip(pattern_str, days):
        if char == '1':  # 1 means working
            named_days.append(day_name)
    return ", ".join(named_days)

def analyze_schedules(excel_file: str):
    # 1. Load data
    df = pd.read_excel(excel_file)
    
    # 2. Create a numeric day pattern string
    day_cols = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    df["Day_Combination"] = df[day_cols].astype(str).agg(''.join, axis=1)
    
    # 3. Convert numeric pattern to text
    df["Day_Combination_Text"] = df["Day_Combination"].apply(
        lambda pat: pattern_to_named_days(pat, day_cols)
    )
    
    # 4. Group by Bid Area + the new textual pattern to see how many employees per pattern
    grouped = (
        df.groupby(["Bid Pool Name", "Day_Combination_Text"])["Perner"]
        .count()
        .reset_index()
        .rename(columns={"Perner": "Num_of_CMs"})
    )
    
    # 5. Sort employees by seniority if needed
    df_sorted = df.sort_values(by=["Bid Pool Name", "Seniority Date"])
    
    # 6. Save or print results
    grouped.to_excel("/Users/gduln001/Desktop/day_combinations_by_area.xlsx", index=False)
    df_sorted.to_excel("/Users/gduln001/Desktop/sorted_by_seniority.xlsx", index=False)

    
    print("Analysis complete!")
    print(" - day_combinations_by_area.xlsx created.")
    print(" - sorted_by_seniority.xlsx created.")

if __name__ == "__main__":
    analyze_schedules("GEMS Schedule.xlsx")


Analysis complete!
 - day_combinations_by_area.xlsx created.
 - sorted_by_seniority.xlsx created.
